# <p style="background-color:#34495E; color:lightgray; font-size:110%; text-align:center; border-radius:20px 20px;width:950px;height:50px;line-height:50px;border-style:round;">Python DB API</p>

DB-API is an acronym for DataBase Application Programming Interface and a library that lets Python connect to the database server.

## PyODBC
Pyodbc is a Python DB conformant module for ODBC databases. 

**ODBC** (Open Database Connectivity) is the primary native data access API for applications written in C and C++ for SQL Server. Other languages that can use ODBC include COBOL, Perl, PHP, and Python. ODBC is widely used in data integration scenarios.

In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyodbc

In [2]:
pyodbc.drivers()

['ODBC Driver 18 for SQL Server']

***

## SQL SERVER CONNECTION

### SQL Server Authentication

In [3]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try: 
        conn = pyodbc.connect(DRIVER=driver, SERVER=server, DATABASE=database, UID=user, PWD=password)
        print("MS SQL Server Database connection successful!")
    except Exception as err:
        print(f"ERROR:\t'{err}'")
    return conn

In [7]:
driver = '{ODBC Driver 18 for SQL Server}'
server = 'localhost'
database = 'master'
user = 'sa'          # 'sa' means system administrator
password = 'Kadir1514-'   # your password

conn = create_server_connection(driver, server, database, user, password)

ERROR:	'('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]SSL Provider: [error:0A000086:SSL routines::certificate verify failed:self-signed certificate] (-1) (SQLDriverConnect)')'


In [ ]:
conn.close()  # to close the sql server connection

### Windows Authentication

In [ ]:
conn_string = """driver={ODBC Driver 17 for SQL Server}; 
                 server=localhost; 
                 database=master; 
                 TRUSTED_CONNECTION=yes;"""

In [ ]:
conn = pyodbc.connect(conn_string);

***

## Connection Objects

**cursor :** to access the data in the database
    
**commit :** This method commits the current transaction. It allows the transactions made here to be reflected (commit) to the sql server. If you do not commit, they will not be reflected.

**connection.close :** to close the sql server connection

### Cursor process

In [ ]:
crs = conn.cursor()

In [ ]:
conn.autocommit = True 

In [ ]:
#conn.commit() -- for desired transactions

***

## Cursor Objects

### Create Database

In [ ]:
# database olusturma yontemi 1
crs.execute('CREATE DATABASE TestA')

In [ ]:
# dbase olusturma yontem 2
def create_database(conn, create_db_query):   
    crs = conn.cursor() 
    try:
        crs.execute(create_db_query)  #prepares and executes SQL
        print("Database is Created Successfully")
    except Exception as err:
            print(f"ERROR:\t'{err}'")

In [ ]:
# yontem 2de variablea dbase ismini atariz
create_db_query = 'CREATE DATABASE TestB'

In [ ]:
# yöntem iki son islemi function calistir
create_database(conn, create_db_query)

In [ ]:
# silmek icin
crs.execute('DROP DATABASE TestB')

### Executing a query

All SQL statements are executed using the Cursor execute() function. 

In [ ]:
# sql'e yazacagimiz herseyi bu sekilde buraya yaziyor ve crs.execute() ile calistiriyoreuz
query = 'USE TestA'

In [ ]:
crs.execute(query)

In [ ]:
# ya da bunu function ile calistiririz. bu functionu veya yukardakini kullanabiliriz farketmez
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Exception as err:
        print(f"ERROR:\t'{err}'")

In [ ]:
execute_query(conn,query)

### Create Table

In [ ]:
query = """CREATE TABLE TestTable (
                ID INT IDENTITY (1,1) NOT NULL, 
                FirstName VARCHAR(255) NOT NULL,
                LastName VARCHAR(255) NOT NULL,
                PRIMARY KEY(ID))"""

In [ ]:
execute_query(conn, query)

### Insert Values

In [ ]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [ ]:
# parameterized
# parametre kullanmak icin. hangi degerleri atayacagimiz belli degilse her deger icin bir ? koyarız ve virgülle ayırırız.
# özellikle baska tablodan veri cektigimizde bu yontem cok kullanıslı
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Bruce', 'Lee')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", ('Jerry', 'Blue'))
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", ('Katie', 'Holmes'))

In [ ]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Tom', 'Cat')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Jerry', 'Mouse')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Owen', 'William')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Stefan', 'Müller')

In [ ]:
#crs.execute("SELECT * FROM TestTable").fetchall()

### Updating and Deleting with rowcount

In [ ]:
crs.execute("DELETE FROM TestTable WHERE FirstName = 'Katie'")
crs.rowcount  # bu kac satir uzerinde islem yaptigimizi gosterir. ornegin katieyi silerse 1 getirir.

In [ ]:
deleted = crs.execute("DELETE FROM TestTable WHERE FirstName = ?", 'Jerry').rowcount
print(deleted, 'row(s) deleted')
# bu sefer variablaesa atayarak yaptik. rowcountin degisik kullanımı
# 2 satır sildi 2 döndü

In [ ]:
# update de ayni mantik: id'si 8 olanin ad ve soyadini natalie portman yap
crs.execute("UPDATE TestTable SET FirstName='Natalie', LastName='Portman' WHERE ID=8")
print(crs.rowcount, 'row(s) updated')

In [ ]:
#crs.execute("SELECT * FROM TestTable").fetchall()

***

## Cursor Functions -- Reading Data

All SQL statements are executed using the Cursor execute() function. 

If the statement returns rows, such as a select statement, you can retrieve them using the Cursor fetch functions - fetchone(), fetchall(), fetchmany(). 

If there are no rows, fetchone() will return None, whereas fetchall() and fetchmany() will both return empty lists.

#### Fetchone

In [ ]:
# buraya kadar hep sql'den okuma yaptik, yukardaki islemlerin sonuclarini sqlden gorduk yani
# python'dan okumak icin fetch
crs.execute('SELECT FirstName, LastName FROM TestTable')

In [ ]:
row = crs.fetchone()  # it returns the first record of the table
row

In [ ]:
crs.fetchone()  # it returns the next record of the table

The `fetchone()` function returns None when all rows have been retrieved.

In [ ]:
crs.execute('select FirstName, LastName from TestTable')
row = crs.fetchone()

print('LastName:', row[1])         # access by column index (zero-based)
print('LastName:', row.LastName)   # access by name

In [ ]:
# while loop ile tüm isimleri getirebiliriz istersek
crs.execute('select FirstName, LastName from TestTable')
while True:
    row = crs.fetchone()
    if not row:
        break
    print('FirstName:', row.FirstName)

#### Fetchmany

In [ ]:
# yanına paramtere alir ve o parametre kadar getirir.
crs.execute("""SELECT ID, FirstName, LastName 
                FROM TestTable""")
rows = crs.fetchmany(3)  # 3 tanesini getir
rows

In [ ]:
rows[0]  # indeksleme

#### Fetchall

In [ ]:
# onemli olan ve asıl kullanılan bu. query'nin tum sonucunu getirir.
# sql ile aynı mantık.
crs.execute('SELECT ID, FirstName, LastName FROM TestTable')
rows = crs.fetchall()
rows 

In [ ]:
# for loop ile istersek tek tek de getirebiliriz.
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

***

## Connecting Different Databases

In [ ]:
# SampleREtail dbase'e gidelim ve onu kullanalım
crs.execute('USE SampleRetail')

In [ ]:
crs.execute("SELECT * FROM product.category").fetchall()

In [ ]:
query = """SELECT  
                RIGHT(email, LEN(email)-CHARINDEX('@', email)) AS DomainType,
                COUNT(RIGHT(email, LEN(email)-CHARINDEX('@', email))) AS NumofDomains
           FROM 
                sale.customer
           GROUP BY
                RIGHT(email, LEN(email)-CHARINDEX('@', email))
           ORDER BY
                NumofDomains DESC;"""

In [ ]:
crs.execute(query).fetchall()

***

## Pandas Methods

### Create Dataframe

In [ ]:
# bunları nasıl dataframe'e donusturecegiz. df yapip tum python islemlerini uygulayabiliriz

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
query = 'USE TestA'
execute_query(conn,query)

In [ ]:
crs.execute('SELECT * FROM TestTable')
results = crs.fetchall()
results 

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df

In [ ]:
# ayrı sutunlara bolmek
from_db = []

for result in results:
    result = list(result)
    from_db.append(result)

In [ ]:
from_db

In [ ]:

cols = ['ID','FirstName','LastName']
df = pd.DataFrame(from_db, columns = cols)
df

***

In [ ]:
query = 'USE SampleRetail'
execute_query(conn,query)

In [ ]:
# for loops yerine bunun daha kolay yontemi
df_prod = pd.read_sql("SELECT * FROM product.product", con=conn)  # ikinci conn bizim atadigimiz variable ismi
df_prod.head(10)

# artik burda cikanlari her turku visualization islemine sokabiliriz.

In [ ]:
# elimizde bir excel sayfasi var ve onu sql databasei yapmak istersek:

query = """
        CREATE TABLE product_new1 (
            product_id int,
            product_name varchar(255),
            brand_id int,
            category_id int ,
            model_year int,
            list_price decimal(10,2)
        )
        """
execute_query(conn, query)

In [ ]:
# bunu dictionary formatina atayalim. records pandasın kendi parametresi. liste icinde her satırı bir dictionary icine 
# atar
df_prod.to_dict('records')

In [ ]:
# bunu bir degiskene atayalim
records = df_prod.to_dict('records')

In [ ]:
# o an hangi degeri kullanacagimizi bilmedigimiz icin ? kullanalım. 6 sutun icin 6 adet ?
sql = 'INSERT product_new1 VALUES(?,?,?,?,?,?)'

In [ ]:
# aslında product_id indexi oldugu icin bu tablo icin enumerate'e gerek yok. ama olur da id-index yoksa enumerate ile
# identity id olustururuz. 
for k,v in enumerate(records, start=1):
    print(k,v)
    
# dictionary mantığı sql e daha uygun olduğu için dict kullanmak daha etkin

In [ ]:
# bunu list olarak da yapabilirdik, ama listede isimle indeksleme yapamiyoruz. 
# bu kod her satırdaki sadece prod nameleri getirir. toplam 520 isim
for k,v in enumerate(records):
    print(v['product_name'])

In [ ]:
# bunu insert ediyoruz. yukarıda insert icin sql variable tanımlamistik
# sql = 'INSERT product_new1 VALUES(?,?,?,?,?,?)'
for k,v in enumerate(records):
    crs.execute(sql, (v['product_id'], v['product_name'], v['brand_id'], v['category_id'], 
                      v['model_year'], v['list_price']))

In [ ]:
crs.execute("SELECT * FROM product_new1").fetchall()

In [ ]:
# en son isimiz bittiginde close ile kapatiriz.
conn.close()

In [ ]:
# CV icin: 1complex query yazabiliyor
# 2. python ile gorsellestirebiliyor